PomodoroAutoTracker

In [ ]:
from IPython.display import display, clear_output
from ipywidgets import VBox, HBox, Layout, Output, FloatProgress, ToggleButtons, Text, Button, Tab, Checkbox, Valid
from threading import Timer # start() cancel()
from datetime import timedelta, datetime
from time import sleep
from pandas import DataFrame, DatetimeIndex
import qgrid
from os.path import isfile
import simpleaudio as sa

In [ ]:
statuses = ['work','longBreak','shortBreak']
cs = dict(zip(statuses,[0]*3))
cs['pause']=0
state, wait, when = 'shortBreak', 10, datetime.now()+timedelta(seconds=10)
ttt, dnt, pst = None, None, None
statusTextA, statusTextB = None, None

In [ ]:
floatProgress = FloatProgress( description='FloatProgressFloatProgress',
                                  style = {'description_width': 'initial'} , layout=Layout(width='40%'))
toggleButtons = ToggleButtons( options=['Start ', 'Pause ', 'Stop '], index=2, description='', 
                                  icons=['play', 'pause', 'stop'], disabled=False, button_style='', layout=Layout(width='40%'))

snoozeButton = Button( description='Add a minute', tooltip='Delay next log', icon='history',
                        style = {'description_width': 'initial'}, layout=Layout(width='100%')) #calendar-plus-o
nextButton = Button( description='Finish early', tooltip='Log & skip to next', icon='fast-forward',
                       style = {'description_width': 'initial'}, layout=Layout(width='100%')) #check
statusText = Text( description='Status', placeholder=' Current -> Next, Waiting Time, Event counters', layout=Layout(width='40%'))#style = {'description_width': 'initial'},
nextLogText = Text( description='Logging next', placeholder=' Activity - Task - Comment - Success | Break', layout=Layout(width='40%'))#style = {'description_width': 'initial'},


# setup times
df_times = DataFrame( data=[[24,'Invincible'],[18,'SexyWays'],[3,'BattleAgainstClown']], index=statuses, columns=['Duration(min)', 'FileName(.wav)']) #, dtype=int 90,60,30
df_times.index.set_names('indexName', inplace=True)
qw_times = qgrid.show_grid( df_times, show_toolbar=False)
playSoundCheckbox = Checkbox(value=False, description='Play audio' , style = {'description_width': 'initial'} , layout=Layout(width='150px'))
playSoundValid = Valid(value=False, description='File check:', style = {'description_width': 'initial'} , layout=Layout(width='200px'))

# activities
df_todo = DataFrame( data=[['Planning','default task','commit message', True],
                             ['Chores'  ,'Dishes'      ,'hate it!', False]], 
                     columns=['Activity','Task','Comment', 'Success'] )
df_todo.index.set_names('indexName', inplace=True)
qw_todo = qgrid.show_grid( df_todo, show_toolbar=True, column_definitions={'indexName':{'width':37},'Success':{'width':16}})
# log
df_log = DataFrame( data=[['App','Started','Hello world!', True ]], 
                     index=DatetimeIndex([datetime.now()], name='DatetimeIndex'),
                     columns=['Activity','Task','Comment','Success'] )
qw_log = qgrid.show_grid( df_log, show_toolbar=False, column_definitions={'DatetimeIndex':{'width':40},'Success':{'width':16}})
# achievements
outputA = Output()
outputB = Output()
achievementsButton = Button( description='View achievements', tooltip='Count and sum cumulative time', icon='flag-checkered',
                            style = {'description_width': 'initial'}, layout=Layout(width='100%'))

# layout
upperbox = VBox([ HBox([ toggleButtons, floatProgress ]),
                  HBox([ nextButton   , snoozeButton  , achievementsButton ], layout=Layout(margin='0px 10px 0px 2px', width='36%', justify_content='space-between')),
                  HBox([ statusText   , nextLogText   ])  ])
tab = Tab(children = [qw_todo, 
                      qw_log, 
                      VBox([ HBox([ playSoundCheckbox, playSoundValid], layout=Layout(margin='0px 10px 0px 50px', justify_content='flex-start')), qw_times]) ]) #center
#margin='top, right, bottom, left'
tab.set_title(0, 'Activities')
tab.set_title(1, 'Log')
tab.set_title(2, 'TimeSetup')
VBox([ upperbox, tab, HBox([outputA,outputB], layout=Layout(justify_content='space-around')) ])

In [ ]:
def selection_changed_handler(event, qgrid_widget):
    if event['name'] == 'selection_changed' and state=='work':
        sr = event['new'][0]
        nextLogText.value = str(qw_todo.get_changed_df().loc[sr].tolist())

def display_tick(wait, when):
    #print('display_tick', end=';')
    global floatProgress
    delta = when - datetime.now()
    c = 100 * ( 1 - delta.total_seconds() / wait )
    if c < 15:
        floatProgress.bar_style = 'success' #green
    elif c < 50:
        floatProgress.bar_style = '' #blue
    elif c < 70:
        floatProgress.bar_style = 'info' #light blue
    elif c < 85:
        floatProgress.bar_style = 'warning' #orange
    else:
        floatProgress.bar_style = 'danger' #red
    floatProgress.description = state.upper()+' '+str(delta).split('.')[0] 
    floatProgress.value = c

def tick_timer( wait, when):
    #print('tick_timer', end=';')
    global ttt
    if when > datetime.now():
        display_tick( wait, when)
    if when+timedelta(seconds=3.0) > datetime.now():
        ttt = Timer(3.0, tick_timer, args=( wait, when)) #Timer(interval, function, args=None, kwargs=None) start, cancel
        ttt.start()

def getNext(state, cs):
    cs[state]+=1
    if state=='work':
        if cs[state]%4 == 0:
            state='longBreak'
        else:
            state='shortBreak'
    elif state=='shortBreak' or 'longBreak':
        state='work'
    return state, cs

def doNext():
    #print('do next', end=';')
    global state, cs, wait
    log()
    oldState=state
    state, cs = getNext(state, cs)
    print('doNext: %s->%s'%(oldState,state), end=';')
    wait=int(qw_times.get_changed_df().loc[state,'Duration(min)']*60)
    waitFor(wait)
    
def waitFor(wait):
    print('waitFor', wait, end=';')
    global when, dnt, pst
    when = datetime.now()+timedelta(seconds=wait)
    nextState, _ = getNext(state, cs)
    statusText.value = 'Current: {}, next: {}, duration: {}s, counter:{}'.format(state, nextState, wait, cs)
    nextLogText.value = state if state!='work' else str(qw_todo.get_changed_df().loc[qw_todo.get_selected_rows()[0]].tolist())
    dnt = Timer(wait, doNext)
    dnt.start()
    if playSoundValid.value:
        pst = Timer( wait - 30 - qw_times.df.length.loc[ nextState ] , 
                    lambda x,y: qw_times.get_changed_df().wave_obj.loc[x].play() if y.value else print('shhh!') ,
                    args=(nextState, playSoundCheckbox,) )
        pst.start()
    tick_timer(wait, when)

def stopAll():
    print('stopAll', end=';')
    global ttt, dnt, pst
    if dnt!=None and dnt.is_alive():
        dnt.cancel()
        dnt.join()
    if ttt!=None and ttt.is_alive():
        ttt.cancel()
        ttt.join()
    if pst!=None and pst.is_alive():
        pst.cancel()
        pst.join()
    sleep(0.1)

def log():
    global qw_log
    sr = qw_todo.get_selected_rows()[0]
    if state=='work':
        qw_log.add_row( [ ('DatetimeIndex',datetime.now()) ]+ list(qw_todo.get_changed_df().loc[sr].items()) )
        qw_todo.edit_cell(sr, 'Success', False)
    else:
        qw_log.add_row( [ ('DatetimeIndex',datetime.now()), ('Activity',state), ('Task',state),('Comment',cs[state]), ('Success',None) ] )
    
def toggleButtonsHandler(event):
    print('toggleButtonsHandler', end=';')
    global wait, when, state
    if event['name'] == 'label':
        #print([ (k,event[k]) for k in event.keys()])
        if event['new'] == 'Start ' and event['old']!='Pause ':
            print('button start')
            doNext()
            
        elif event['new'] == 'Pause ':
            print('button pause')
            stopAll()
            wait = int((when - datetime.now()).total_seconds())
            log()

        elif event['new'] == 'Start ' and event['old']=='Pause ':
            print('button REstart')
            oldState=state
            state='pause'
            cs[state]+=1
            log()
            state = oldState
            waitFor(wait)
            
        elif event['new'] == 'Stop ':
            print('button stop')
            stopAll()

def snoozeButtonHandler(event):
    print('snoozeButtonHandler')
    global wait
    stopAll()
    wait = int((when - datetime.now()).total_seconds() + 60 )
    waitFor(wait)
        
def nextButtonHandler(event):
    print('nextButtonHandler')
    global wait
    stopAll()
    doNext()

def achievementsHandler(event):
    global outputA, outputB
    if len(qw_log.get_changed_df())>4:
        print('achievements', end=';')
        df = qw_log.get_changed_df()
        df['Cumulative'] = df.index.to_series().diff()
        df_A = DataFrame( df[1:].groupby(['Success','Activity','Task']).Cumulative.sum() )
        df_A['Count'] = df[1:].groupby(['Success','Activity','Task']).Cumulative.count()
        #df_A.sort_values('Success', ascending=False, inplace=True)
        df_A.Cumulative = df_A.Cumulative.round('s')
        df_B = DataFrame( df[1:].groupby(['Activity','Task']).Cumulative.sum() )
        df_B['Count'] = df[1:].groupby(['Activity','Task']).Cumulative.count()
        df_B.Cumulative = df_B.Cumulative.round('s')
        with outputA:
            clear_output()
            display(df_A)
        with outputB:
            clear_output() #wait=True
            display(df_B)
            #print('Updated summary @', datetime.now().strftime('%X'))

def waveLenghtAndObject(name):
    wave_read = sa.wave.open(name+'.wav', 'rb') #
    nf, fr = wave_read.getnframes(), wave_read.getframerate()
    wave_obj = sa.WaveObject.from_wave_read(wave_read)
    seconds = nf / fr
    return seconds, wave_obj

def playSoundCheckboxHandler(event):
    global playSoundCheckbox, qw_times
    if event.name == 'value':
        print('playSoundCheckboxHandler', end=',')
        if event.new:
            playSoundValid.value = False
            print('checking audio', end=',')
            df = qw_times.get_changed_df()
            df['exists'] = df[ 'FileName(.wav)' ].apply(lambda x: isfile(x+'.wav'))
            df['length'] = df[ df.exists == True ]['FileName(.wav)'].apply( waveLenghtAndObject )
            df['wave_obj']=df[ df.exists == True ].length.apply(lambda x: x[1])
            df['length'] = df[ df.exists == True ].length.apply(lambda x: round(x[0],1) )
            if not df.exists.all():
                print('File error check, impending DOOM!')
                display(df[ df.exists == False ])
                playSoundValid.value = False
            if df['Duration(min)'].min()*60 < df.length.max()+30:
                print('Audio file longer than pomodoro interval +30s, impending DOOM!')
            # TODO check min duration * 60 + 30 > max length
            qw_times.df = df
            playSoundValid.value = True
            playSoundCheckbox.description = 'Play audio'
            print('check ok', end=',')
        else:
            playSoundCheckbox.description = 'Audio off'
            print('audio disabled', end=',')

qw_todo.on('selection_changed', selection_changed_handler)
playSoundCheckbox.observe(playSoundCheckboxHandler)
toggleButtons.observe(toggleButtonsHandler)
snoozeButton.on_click(snoozeButtonHandler)
nextButton.on_click(nextButtonHandler)
achievementsButton.on_click(achievementsHandler)

# Restart Kernel and Run up to this next cell to start using

In [ ]:
playSoundCheckbox.value = True
qw_todo.change_selection([0])

# Lazy implemented

## Export settings

In [ ]:
qw_log.get_changed_df().to_csv('log.csv')
qw_times.get_changed_df().to_csv('setup.csv')
qw_todo.get_changed_df().to_csv('todo.csv')

## Import settings

### preview

In [ ]:
from os.path import isfile
from pandas import read_csv
filenames=['todo.csv','log.csv','setup.csv']
for filename in filenames:
    if isfile(filename):
        df=read_csv(filename)
        display(df)

### import

In [ ]:
from os.path import isfile
from pandas import read_csv, to_datetime
filenames=['todo.csv','log.csv','setup.csv']
for filename in filenames:
    if isfile(filename):
        df=read_csv(filename)
        if filename=='log.csv':
            df.set_index('DatetimeIndex', inplace=True)
            df.index = to_datetime(df.index)
            qw_log.df = df
        else:
            df.set_index('indexName', inplace=True)
            if filename=='todo.csv':    
                qw_todo.df = df
            if filename=='setup.csv':
                playSoundCheckbox.value = False
                qw_times.df = df
                playSoundCheckbox.value = True
                sleep(0.1)